In [1]:
from IPython.display import display, Markdown
from dotenv import load_dotenv as env
from tqdm.auto import tqdm
import pandas as pd
import pickle
import openai

OUTPUT_PATH = './outputs/respuestas_gpt35_turbo.csv'

# Usa 'ChatCompletion' que está deprecado en la v1.
def get_completion(prompt: str, model="gpt-3.5-turbo") -> str:
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def get_completion(*args, **kwargs):
    from random import choice
    return choice(['No sabe / No contesta', 'No preguntada', 'No aplicable', 'No contesta', 'No sabe', 'SÃ\xad', 'No'])

In [2]:
openai.api_key = env['OPENAI_KEY']

In [3]:
with open('./datasets/preguntas_seleccionadas.pickle', 'rb') as handle:
    preguntas_seleccionadas: dict[str, str] = pickle.load(handle)

with open('./datasets/respuestas.pickle', 'rb') as handle:
    respuestas: dict[str, str] = pickle.load(handle)

In [4]:
respuestasGPT = {}

preguntas = list(preguntas_seleccionadas.keys())

OPINION_STR = '. Si no tiene opinión responda "No responde"'
MULTIPLE_CHOICE_STR = '  Responda alguna de las siguientes opciones, si no tiene opinión responda "No responde": \n'

for pregunta in tqdm(preguntas):
    options = OPINION_STR if respuestas[pregunta][1] == 1 \
                else MULTIPLE_CHOICE_STR + ' \n'.join(respuestas[pregunta].values())

    response = get_completion(preguntas_seleccionadas[pregunta]+options)
    response = response.replace('.','').strip()

    inv_map = {v: k for k, v in respuestas[pregunta].items()}
    
    respuestasGPT[pregunta] = inv_map[response] if response in inv_map.keys() else -2

    preguntas = preguntas[len(respuestasGPT):]

  0%|          | 0/80 [00:00<?, ?it/s]

In [5]:
df_GPT_3 = pd.DataFrame.from_dict(respuestasGPT, orient='index').reset_index()
df_GPT_3.columns = ['Pregunta', 'Respuesta']

display(df_GPT_3.head())
display(Markdown('**Conteo por respuesta:**'))
print(df_GPT_3['Respuesta'].value_counts())

,Pregunta,Respuesta
0,P11S0037,-2
1,P11S0038,-2
2,P2ST,-2
3,P12ST,-2
4,P4STGBS,-2


**Conteo por respuesta:**

Respuesta
-2    49
 2    15
 1     6
 3     4
-1     3
 5     1
Name: count, dtype: int64

In [6]:
df_GPT_3.to_csv(OUTPUT_PATH)